<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/CodeBERT_1_N_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install datasets

In [23]:
import pandas as pd

In [46]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/final_train.csv')

In [ ]:
df

In [ ]:
df.columns

In [47]:
df = df[['CODE', 'IDX']]

In [ ]:
df.columns

In [7]:
df

,CODE,IDX
0,void fz_init_cached_color_converter(fz_context...,1
1,"add_range(fz_context *ctx, pdf_cmap *cmap, uns...",10
2,"begin_softmask(fz_context *ctx, pdf_run_proces...",10
3,"add_range(fz_context *ctx, pdf_cmap *cmap, uns...",10
4,"pdf_show_image(fz_context *ctx, pdf_run_proce...",1
...,...,...
24237,"adisplay_draw_glyph( void* _display,\...",0
24238,int reds_handle_migrate_data(MainChannelClient...,0
24239,static ssize_t fuse_direct_read(struct file *f...,0
24240,static int __init qeth_core_init(void)\n{\n\ti...,0


In [48]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [49]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    preprocess_df = df.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df = preprocess_df.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df = preprocess_df.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df = preprocess_df.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df = df.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['CODE'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [51]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/24242 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [68]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=200)

In [69]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [70]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [55]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics


In [74]:
training_args = TrainingArguments(
    output_dir='./results',             # 학습 결과를 저장할 디렉터리
    num_train_epochs=7,                 # 데이터셋 크기에 맞춰 조정된 학습 에폭 수
    per_device_train_batch_size=16,     # 훈련 배치 크기, GPU 메모리에 맞게 조정 가능
    per_device_eval_batch_size=64,      # 평가 배치 크기, GPU 메모리에 맞게 조정 가능
    warmup_steps=500,                   # 워밍업 스텝 수, 총 스텝의 10%를 고려하여 조정 가능
    weight_decay=0.01,                  # 가중치 감소
    evaluation_strategy="epoch",        # 에폭마다 평가 수행
    logging_dir='./logs',               # 로그를 저장할 디렉터리
)

trainer = Trainer(
    model=model,                         # 학습시킬 모델
    args=training_args,                  # 학습 인자
    train_dataset=encoded_dataset['train'],         # 훈련 데이터셋
    eval_dataset=encoded_dataset['test'],           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 메트릭 계산 함수
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [57]:
gc.collect()
torch.cuda.empty_cache()

In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9,F1 Class 10
1,0.607000,0.471055,0.836931,0.836931,0.832849,0.794728,0.836931,0.944683,0.553926,0.548148,0.999059,0.988142,0.986355,1.000000,0.587037,0.882907,0.613372,0.638376
2,0.353300,0.351186,0.896329,0.896329,0.895611,0.873437,0.896329,0.960683,0.766881,0.681402,1.000000,0.980315,0.984221,1.000000,0.765403,0.919481,0.746586,0.802837
3,0.224100,0.291714,0.918603,0.918603,0.917728,0.899671,0.918603,0.968662,0.832025,0.737662,1.000000,0.990020,0.990329,1.000000,0.816970,0.931472,0.802097,0.827145
4,0.137900,0.308227,0.931940,0.931940,0.931350,0.917126,0.931940,0.975030,0.856489,0.776632,1.000000,0.974516,0.976303,1.000000,0.854214,0.950192,0.857542,0.867470
5,0.090800,0.340490,0.937990,0.937990,0.937572,0.924863,0.937990,0.974307,0.867659,0.814751,1.000000,0.989184,0.989205,1.000000,0.840317,0.960688,0.847278,0.890110
6,0.045400,0.330809,0.944177,0.944177,0.943604,0.932679,0.944177,0.977328,0.886551,0.810435,1.000000,0.986111,0.986381,1.000000,0.872093,0.965602,0.880785,0.894180
7,0.029200,0.346499,0.945552,0.945552,0.945048,0.934586,0.945552,0.977149,0.888213,0.821918,1.000000,0.988072,0.988350,1.000000,0.866123,0.961071,0.889819,0.899729


TrainOutput(global_step=7427, training_loss=0.2605690002698219, metrics={'train_runtime': 3978.9936, 'train_samples_per_second': 29.853, 'train_steps_per_second': 1.867, 'total_flos': 3.1255645971250176e+16, 'train_loss': 0.2605690002698219, 'epoch': 7.0})

In [71]:
trainer.evaluate()

{'eval_loss': 0.34649887681007385,
 'eval_accuracy': 0.9455520417984326,
 'eval_recall_weighted': 0.9455520417984326,
 'eval_f1_weighted': 0.9450482985556065,
 'eval_f1_macro': 0.9345856455713067,
 'eval_f1_micro': 0.9455520417984326,
 'eval_f1_class_0': 0.9771486349848332,
 'eval_f1_class_1': 0.8882129277566541,
 'eval_f1_class_2': 0.821917808219178,
 'eval_f1_class_3': 1.0,
 'eval_f1_class_4': 0.9880715705765407,
 'eval_f1_class_5': 0.9883495145631068,
 'eval_f1_class_6': 1.0,
 'eval_f1_class_7': 0.8661233993015134,
 'eval_f1_class_8': 0.9610705596107055,
 'eval_f1_class_9': 0.8898186889818689,
 'eval_f1_class_10': 0.899728997289973,
 'eval_runtime': 68.591,
 'eval_samples_per_second': 106.034,
 'eval_steps_per_second': 1.662,
 'epoch': 7.0}

In [60]:
trainer.save_model('model')

In [80]:
import pandas as pd

df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model train/final_val.csv")

In [81]:
df2 = df2[['CODE', 'IDX']]

In [82]:
df2

,CODE,IDX
0,nautilus_file_mark_desktop_file_trusted (GFile...,1
1,"int Track::Info::CopyStr(char* Info::*str, Inf...",1
2,long Tracks::ParseTrackEntry(long long track_s...,1
3,void HttpAuthFilterWhitelist::SetWhitelist(\n...,1
4,void CreateOAuth2ServiceDelegate(\n si...,1
...,...,...
475,static int au1200fb_ioctl(struct fb_info *info...,0
476,"static int nfs_read(struct device_d *dev, FILE...",0
477,static ChannelSecurityOptions *find_channel_se...,0
478,"uiserver_decrypt_verify (void *engine, gpgme_d...",0


In [84]:
import re

def preprocess2(df2, file_name):
    # 멀티 라인 주석 제거
    preprocess_df2 = df2.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df2 = preprocess_df2.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df2 = preprocess_df2.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df2 = df2.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df2.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['CODE'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [101]:
preprocess2(df2,"preprocess2.csv")
dataset = load_dataset("csv",data_files="preprocess2.csv")['train']
encoded_dataset2 = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset2=encoded_dataset2.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [102]:
dataset

Dataset({
    features: ['CODE', 'IDX'],
    num_rows: 480
})

In [103]:
encoded_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 480
})

In [104]:
trainer.predict(encoded_dataset, metric_key_prefix="eval")

PredictionOutput(predictions=array([[-0.9825927 , -1.4848615 , -1.941013  , ..., -0.6301794 ,
        -1.2276725 , -0.9160334 ],
       [-0.7095964 , -0.92573744, -1.08018   , ..., -1.1521306 ,
        -1.2338915 , -2.2850745 ],
       [-1.1053548 , -1.2251264 , -1.8744897 , ..., -0.8858184 ,
        -1.220711  , -0.8044509 ],
       ...,
       [-0.64717394,  9.385332  , -0.19386388, ..., -1.2386785 ,
        -0.83632267, -1.1871543 ],
       [-1.2673211 , -0.40526477, -0.62709045, ..., -1.2998829 ,
        -0.91408736, -0.71725214],
       [ 2.1331809 , -1.0603514 , -0.6636715 , ..., -1.9141536 ,
        -0.73397636,  8.196585  ]], dtype=float32), label_ids=array([ 5,  3,  5,  6,  0,  9,  5,  9,  1, 10,  3,  6,  1,  0,  4,  0,  4,
        7,  0,  0, 10, 10,  6,  0,  1,  4,  3,  5,  9,  0, 10,  6,  0,  3,
        0, 10,  5,  9,  0,  0,  7, 10,  4,  8, 10,  0,  5,  7,  4,  0,  9,
        5,  8,  6,  8,  7,  0, 10,  3,  1,  0,  0,  9,  0,  8,  8,  2,  2,
        8, 10,  9,  0,  0,  0,  